In [18]:
import pickle
import numpy as np
import copy
def save_obj(name, obj):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, protocol = 2)
        
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)#, encoding='latin1')

import healpy as hp

'''
This notebook compute the mixing matrixes (see, e.g., https://arxiv.org/pdf/1603.07818.pdf)
for a given mask.

'''
# load the mask
mask = load_obj("mask_DES_y3_py2")
#save_obj("mask_DES_y3_py2",mask)
print ('f_sky: ', 1./(len(mask)*1./len(mask[mask])))
mask_sm =  hp.sphtfunc.smoothing(mask, (3./60.)*np.pi/180.  )
mask_sm[mask] = 1.
mask = copy.copy(mask_sm)
# computes Cl.
lmax = 2048
alms_mask = hp.map2alm(mask, lmax=lmax)  # Spin transform
Cl_mask =  hp.sphtfunc.alm2cl(alms_mask)

#print ('f_sky: ', 1./(len(mask_sm)*1./np.sum(mask_sm)))

f_sky:  0.11542924245198567
Sigma is 1.273983 arcmin (0.000371 rad) 
-> fwhm is 3.000000 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin


# NaMaster matrix

In [19]:
import h5py

# compute coupling matries with Namaster.

import pymaster as nmt
print('loaded')
# Read mask and apodize it on a scale of ~1deg
#mask = nmt.mask_apodization(mask,1., apotype="Smooth")
#hp.mollview(mask, coord=['G', 'C'], title='Apodized mask')
#plt.show()

# Read healpix maps and initialize a spin-0 and spin-2 field
f_0 = nmt.NmtField(mask, [mask])
f_2 = nmt.NmtField(mask, [mask,mask])
bins = nmt.bins.NmtBin.from_lmax_linear(2048, 1, is_Dell=False)#nmt.bins.NmtBin(nside=1024, ells=2048)


w = nmt.NmtWorkspace()
w.compute_coupling_matrix(f_0, f_0, bins, is_teb=False)
Mgg = w.get_coupling_matrix()
save_obj('Mgg',Mgg)

w = nmt.NmtWorkspace()
w.compute_coupling_matrix(f_2, f_2, bins, is_teb=False)
M = w.get_coupling_matrix()
ME = (M[::4,:][:,::4])


name_file ='/global/project/projectdirs/des/mgatti/Moments_analysis/mode_coupling_matrix_NaMaster_3arcmin_sm_{0}_{1}.h5'.format(lmax,nside)


# you might need to transpose these. check it ***
h5f = h5py.File(name_file, 'w')
h5f.create_dataset('ME', data=(ME[:2048,:][:,:2048]).reshape(2048*2048))
h5f.create_dataset('MB', data=Mgg*0)
h5f.create_dataset('MgE', data=Mgg*0)
h5f.create_dataset('Mgg', data=(Mgg[:2048,:][:,:2048]).reshape(2048*2048))
h5f.close()

loaded


In [3]:

import numpy as np
from numba import double
from numba.decorators import jit
import numpy as np
import wignerpy._wignerpy as wp

'''
This is the function that computes all the mixing matrixes as a function of l1.
'''
def triple_cycle_3(l1,u,lenU):
    '''
    ME = MEE,EE
    MB = MEE,BB
    MgE = MGE,GE (gg lensing)
    Mgg = Mgg,gg (clustering)
     
    '''
    a = 1./(8*np.pi)
    l3 = np.arange(3*lenU)
    mask1 = l3<lenU
    ME = np.zeros(lenU)
    MB = np.zeros(lenU)
    MgE = np.zeros(lenU)
    Mgg = np.zeros(lenU)
    for l2 in range(2,lenU):
 
        
        mask = ((l1-l2)**2 <= l3*l3) & ((l1+l2)>=l3) 
        mask2 = l3[mask]<lenU
        #print len(mask),len(l3), len(u),len(u[mask])
       
        v1 = wp.wigner3jvec(l1,l2, 2,-2)
        v0 = wp.wigner3jvec(l1,l2, 0, 0)

        ME[l2] += np.sum((2*l2+1)*(2*l3[mask&mask1]+1)* a * u[mask[mask1]] * ( 1 + (-1.)**(l1+l2+l3[mask&mask1]))*v1[mask2]**2.)
        MB[l2] += np.sum((2*l2+1)*(2*l3[mask&mask1]+1)* a * u[mask[mask1]] * ( 1 - (-1.)**(l1+l2+l3[mask&mask1]))*v1[mask2]**2.)
        MgE[l2] += 2.*np.sum((2*l2+1)*(2*l3[mask&mask1]+1)* a * u[mask[mask1]] *(v0[mask2]*v1[mask2])**2)
        Mgg[l2] += 2.*np.sum((2*l2+1)*(2*l3[mask&mask1]+1)* a * u[mask[mask1]] *v0[mask2]**2.)

    return ME, MB, MgE, Mgg,l1


In [4]:
import timeit
import numpy as np
from multiprocessing import Pool,sharedctypes
from functools import partial
from contextlib import closing
import h5py
import wignerpy._wignerpy as wp
def _init(arr_to_populate):
    global arr
    arr = arr_to_populate
    
nside=1024
lmax = nside*2
agents = 64


l1 = range(2,lmax)
    

ME_final = np.zeros(lmax*lmax)
MB_final = np.zeros(lmax*lmax)
MgE_final = np.zeros(lmax*lmax)
Mgg_final = np.zeros(lmax*lmax)
t3 = timeit.default_timer()

mute_list = []
l2l = (2*np.arange(lmax)+1)

with closing(Pool(processes=agents)) as pool:
     mute_list.append(pool.map(partial(triple_cycle_3, u = Cl_mask[:lmax], lenU = lmax),l1))
        
        
for i in range(np.array(mute_list).shape[1]):
    ME_final[lmax*(i):lmax*(i+1)] = np.array(mute_list)[0,i,0]
    MB_final[lmax*(i):lmax*(i+1)] = np.array(mute_list)[0,i,1]
    MgE_final[lmax*(i):lmax*(i+1)] = np.array(mute_list)[0,i,2]
    Mgg_final[lmax*(i):lmax*(i+1)] = np.array(mute_list)[0,i,3]
t4 = timeit.default_timer()
print (t4-t3)

name_file ='/global/project/projectdirs/des/mgatti/Moments_analysis/mode_coupling_matrix_sm6arcmin_{0}_{1}.h5'.format(lmax,nside)

h5f = h5py.File(name_file, 'w')
h5f.create_dataset('ME', data=ME_final)
h5f.create_dataset('MB', data=MB_final)
h5f.create_dataset('MgE', data=MgE_final)
h5f.create_dataset('Mgg', data=Mgg_final)
h5f.close()
print "done"

185.960356951
done
